## Tasks

### Faults

* How many occur per day, how many assets are in the system (use historic data), how many assets per tote type?
* What's our data set size (inner join)
* What happens dropping granularity?
* Map faults to Grey-Blue
* Blue faults and availability
* Grey faults and availability
* Fault distribution by tote colour: at top level (Grey Vs. Blue), lower level (faults within colour)
* Faults by asset: TimeAssetIs in Fault ~ Total Availibility
* Do commonly occuring faults have a relationwith time availibility? Are some faults just warnings?
* Faults by hour / shift pattern?

### Active Totes

* Active tote variability by day, hour?
* Actives by hour / shift pattern?
* Need to aggregate by hour - also is there variability within an hour?
* Correlation between active totes and availability - optimum curve

### Blue-Grey Availability data

* How does each availability vary: overall, grey, blue? Does it vary through time period
* Pick Station availability: overall, blue, grey
* Pick station availability compared to active totes: whole SCS, quadrant, module

In [1]:
import pandas as pd
import seaborn as sns

### Raw Files

In [2]:
scs_raw = pd.read_csv('../data/SCS alerts Nov_with_asset_code.csv', parse_dates=['Entry time '])
scs_raw.columns = pd.Series(scs_raw.columns).str.strip()
#scs_raw = scs[~scs['PLC'].isin(['C23','C15','C16', 'C17'])].copy() # check that I can drop these as outside
active_totes = pd.read_csv('../data/active_totes_20201123.csv')
availability = pd.read_csv('../data/Availability_with_Grey&Blue_1811-2511.csv')

### Clean SCS

In [3]:
# Remove destacker
scs_raw['PLC_number'] = scs_raw['PLC'].str.extract('((?<=^C).*)').fillna('0').astype(int) # with regex
scs = scs_raw

### Clean Active Totes

In [4]:
active_totes['MODULE_ASSIGNED'].unique() # scsXX - CXX 05 <= XX <= 14

array(['SCS01', 'SCS02', 'SCS03', 'SCS04', 'SCS05', 'SCS07', 'SCS08',
       'SCS09', 'SCS10', 'SCS11', 'SCS12', 'SCS13', 'SCS14', 'SCS15',
       'SCS17', 'SCS18', 'SCS19', 'SCS20', 'ECB', 'RCB'], dtype=object)

In [5]:
active_totes_drop = active_totes[~active_totes['MODULE_ASSIGNED'].isin(['ECB', 'RCB'])].copy()
active_totes_drop['module_number'] = active_totes_drop['MODULE_ASSIGNED'].str.slice(3,5)
active_totes_drop['module_number'] = active_totes_drop['MODULE_ASSIGNED'].str.extract('((?<=[A-Z]{3}).*)') # with regex
active_totes_drop['module_number'] = active_totes_drop['MODULE_ASSIGNED'].apply(lambda x: x[3::]) # with a lambda

In [6]:
active_totes_drop['DAY'] = active_totes_drop['DAY'].astype('str').str.pad(width=2, side='left', fillchar='0')
active_totes_drop['HOUR'] = active_totes_drop['HOUR'].astype('str').str.pad(width=2, side='left', fillchar='0')
active_totes_drop['MINUTE'] = active_totes_drop['MINUTE'].astype('str').str.pad(width=2, side='left', fillchar='0')
active_totes_drop['timestamp'] = pd.to_datetime(active_totes_drop.apply(
    lambda x: '{0}/{1}/{2} {3}:{4}'.format(x['MONTH'],x['DAY'], x['YEAR'], x['HOUR'], x['MINUTE']), axis=1))

In [7]:
active_totes_drop = active_totes_drop.drop(['ID', 'DAY', 'MONTH', 'YEAR', 'HOUR', 'MINUTE'], axis=1)

In [8]:
active_totes_drop.head()

,MODULE_ASSIGNED,TOTES,module_number,timestamp
0,SCS01,44,01,2020-11-09 08:22:00
1,SCS02,33,02,2020-11-09 08:22:00
2,SCS03,71,03,2020-11-09 08:22:00
3,SCS04,53,04,2020-11-09 08:22:00
4,SCS05,65,05,2020-11-09 08:22:00


### Link Totes and Faults

In [9]:
# tote lookup
lu = pd.read_csv('../data/asset_tote_lookup.csv')
lu = lu[['Name', 'Tote Colour']]

In [10]:
scs_totes = pd.merge(scs_raw, lu, how='left', left_on='code', right_on='Name', indicator=True)
# Big rules
scs_totes.loc[scs_totes['Alert'].str.contains('PTT'), 'Tote Colour'] = 'Both'
scs_totes.loc[(scs_totes['PLC_number'] > 34), 'Tote Colour'] = 'Blue'
scs_totes.loc[scs_totes['PLC_number'].isin([15,16,17,23]), 'Tote Colour'] = 'Blue'

In [11]:
scs_totes['Tote Colour'].value_counts()

Blue    69536
Grey    57770
Both    40052
Name: Tote Colour, dtype: int64

In [13]:
scs_totes.to_csv('../data/scs_tote_matched.csv', index=False)

In [31]:
scs_totes[scs_totes['Tote Colour'].isna() & (scs_totes['Desk']=='Z') & (scs_totes['PLC_number'] >= 5)
         & (scs_totes['PLC_number'] <= 14)]

,Number,Alert,Entry time,PLC,Desk,Duration,Fault ID,code,PLC_number,Name,Tote Colour,_merge
34406,53714,C06: C06 fieldbus summary fault,2020-06-11 02:45:00,C06,Z,0.00:02:56,3107,C06,6,NaN,NaN,left_only
43456,118938,C14: C14+194BV81 fieldbus fault,2020-07-11 12:04:00,C14,Z,0.00:50:30,3104,C14,14,NaN,NaN,left_only
43457,118929,C14: C14+194BV81 circuit breaker monitoring 24VDC,2020-07-11 12:04:00,C14,Z,0.00:50:30,3222,C14,14,NaN,NaN,left_only
43518,119002,C14: C14+143BV83 fieldbus fault,2020-07-11 12:26:00,C14,Z,0.00:00:01,3104,C14,14,NaN,NaN,left_only
43519,118993,C14: C14+143BV83 circuit breaker monitoring 24VDC,2020-07-11 12:26:00,C14,Z,0.00:00:01,3222,C14,14,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...
148653,44498,C05: C05 fieldbus summary fault,2020-11-20 10:32:00,C05,Z,NaN,3107,C05,5,NaN,NaN,left_only
156971,79490,C09: C09+D02_ESA02 emergency stop button actua...,2020-11-23 07:52:00,C09,Z,0.00:00:54,3436,C09,9,NaN,NaN,left_only
157070,79490,C09: C09+D02_ESA02 emergency stop button actua...,2020-11-23 07:37:00,C09,Z,0.00:04:01,3436,C09,9,NaN,NaN,left_only
161671,93643,C11: C11 emergency stop monitoring area 10,2020-11-24 16:26:00,C11,Z,0.00:00:17,3211,C11,11,NaN,NaN,left_only


### Unmatched 3222 & 3114

In [24]:
scs_totes.iloc[(scs_totes['Desk'].isin(['Z']) & (scs_totes['PLC_number'] >= 5) & (scs_totes['PLC_number'] <= 14) &
         scs_totes['Tote Colour'].isna()), 'Tote Colour'] = 'Both'

NotImplementedError: iLocation based boolean indexing on an integer type is not available